In [ ]:
!unzip gender-based-violence-tweet-classification.zip

Archive:  gender-based-violence-tweet-classification.zip
  inflating: StarterNotebook.ipynb   
  inflating: SampleSubmission.csv    
  inflating: Test.csv                
  inflating: Train.csv               


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 14.0 MB/s 
     |████████████████████████████████| 3.3 MB 66.5 MB/s 
     |████████████████████████████████| 895 kB 53.0 MB/s 
     |████████████████████████████████| 636 kB 46.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.data import Dataset
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification
from transformers import logging

logging.set_verbosity_warning()

pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 1000)


In [ ]:
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

np.seed = 200
train_df = train_df.sample(frac=1, random_state=200)
train_df.head()

,Tweet_ID,tweet,type
23638,ID_LK3FHU9H,They said something along the lines of this “ i know him he wouldn’t do something like that “ as well as telling one of my bestfriends that they didn’t believe me because i “ just wanted dick” and i wouldn’t want it if i was raped. Men suck.,sexual_violence
18880,ID_H6OK5R5D,"god did fuck all to do with children starving and people getting raped and murdered for thousands of years, and you say he created me? how about he creates a happy planet with good people, fuck your god he can eat shit",sexual_violence
34891,ID_VRRYJ4Z3,"oh, let us count the questions that Donnie Trump must answer. 1) How does he get by paying $0 to $750 in income taxes when most average Americans pay about 10 times that in any given year. 2) How many women has Donnie raped?",sexual_violence
1015,ID_0VJU7B2W,"FGM(Female Genital Mutilation) includes the partial/total removal of the female external genitalia, causing other injuries to the female genital organs for non-medical reasons. 'Why I broke the law to undergo FGM aged 26'",Harmful_Traditional_practice
28350,ID_PUR1YBUB,My husband always beats me at Scrabble.,Physical_violence


In [ ]:
def replace_hidden(tweet):
    hidden_dic = {'p****': 'pussy',
                  'p*****': 'pussy',
                  'p******': 'pussy',
                  'p***': 'pussy',
                  'p**': 'pussy',
                  'p*': 'pussy',
                  'p****y': 'pussy',
                  'p*****y': 'pussy',
                  'p******y': 'pussy',
                  'p***y': 'pussy',
                  'p**y': 'pussy',
                  'p*y': 'pussy',
                  'pu****': 'pussy',
                  'pu*****': 'pussy',
                  'pu******': 'pussy',
                  'pu***': 'pussy',
                  'pu**': 'pussy',
                  'pu*': 'pussy',
                  'pu****y': 'pussy',
                  'pu*****y': 'pussy',
                  'pu******y': 'pussy',
                  'pu***y': 'pussy',
                  'pu**y': 'pussy',
                  'pu*y': 'pussy',
                  'f****king': 'fucking',
                  'f***king': 'fucking',
                  'f**king': 'fucking',
                  'f*king': 'fucking',
                  'f**ing': 'fucking',
                  'f***ing': 'fucking',
                  'f****ing': 'fucking',
                  'fuckin': 'fucking',
                  'f**g': 'fucking',
                  'f***g': 'fucking',
                  'f****g': 'fucking',
                  '**ing': 'fucking',
                  'f*********': 'fuck',
                  'f********': 'fuck',
                  'f*******': 'fuck',
                  'f******': 'fuck',
                  'f*****': 'fuck',
                  'f****': 'fuck',
                  'f***': 'fuck',
                  'f**': 'fuck',
                  'f*': 'fuck',
                  'f*********k': 'fuck',
                  'f********k': 'fuck',
                  'f*******k': 'fuck',
                  'f******k': 'fuck',
                  'f*****k': 'fuck',
                  'f****k': 'fuck',
                  'f***k': 'fuck',
                  'f**k': 'fuck',
                  'f*k': 'fuck',
                  'fu*********k': 'fuck',
                  'fu********k': 'fuck',
                  'fu*******k': 'fuck',
                  'fu******k': 'fuck',
                  'fu*****k': 'fuck',
                  'fu****k': 'fuck',
                  'fu***k': 'fuck',
                  'fu**k': 'fuck',
                  'fu*k': 'fuck',
                  'fu*********': 'fuck',
                  'fu********': 'fuck',
                  'fu*******': 'fuck',
                  'fu******': 'fuck',
                  'fu*****': 'fuck',
                  'fu****': 'fuck',
                  'fu***': 'fuck',
                  'fu**': 'fuck',
                  'fu*': 'fuck',
                  'f*****ed': 'fucked',
                  'f****ed': 'fucked',
                  'f***ed': 'fucked',
                  'f**ed': 'fucked',
                  'f*ed': 'fucked',
                  'f*****d': 'fucked',
                  'f****d': 'fucked',
                  'f***d': 'fucked',
                  'f**d': 'fucked',
                  'f*d': 'fucked',
                  'd**k': 'dick',
                  'd***k': 'dick',
                  'd**': 'dick',
                  'd*': 'dick',
                  'd***': 'dick',
                  'd****': 'dick',
                  'd*****': 'dick',
                  'd******': 'dick',
                  'sh*****': 'shit',
                  'sh****': 'shit',
                  'sh***': 'shit',
                  'sh**': 'shit',
                  'sh*': 'shit',
                  's*****': 'suck',
                  's****': 'suck',
                  's***': 'suck',
                  's**': 'suck',
                  's*': 'suck',
                  's*****t': 'shit',
                  's****t': 'shit',
                  's***t': 'shit',
                  's**t': 'shit',
                  's*t': 'shit',
                  'b*******': 'bitch',
                  'b******': 'bitch',
                  'b*****': 'bitch',
                  'b****': 'bitch',
                  'b***': 'bitch',
                  'b**': 'bitch',
                  'b****ch': 'bitch',
                  'b*****ch': 'bitch',
                  'b***ch': 'bitch',
                  'b**ch': 'bitch',
                  'g**********': 'gang',
                  'g*********': 'gang',
                  'g********': 'gang',
                  'g*******': 'gang',
                  'g******': 'gang',
                  'g*****': 'gang',
                  'g****': 'gang',
                  'g***': 'gang',
                  'g**': 'gang',
                  'g*': 'gang',
                  'n******er': 'nigger',
                  'n*****er': 'nigger',
                  'n****er': 'nigger',
                  'n***er': 'nigger',
                  'n**er': 'nigger',
                  'n*er': 'nigger',
                  'n********': 'nigger',
                  'n*******': 'nigger',
                  'n******': 'nigger',
                  'n*****': 'nigger',
                  'n****': 'nigger',
                  'n***': 'nigger',
                  'n**': 'nigger',
                  'n*': 'nigger',
                  'ni*a': 'nigger',
                  'ni**a': 'nigger',
                  'ni***a': 'nigger',
                  'ni****a': 'nigger',
                  'ni****a': 'nigger',
                  'n*a': 'nigger',
                  'n**a': 'nigger',
                  'n***a': 'nigger',
                  'n****a': 'nigger',
                  'n****a': 'nigger',
                  'n*******r': 'nigger',
                  'n******r': 'nigger',
                  'n*****r': 'nigger',
                  'n****r': 'nigger',
                  'n***r': 'nigger',
                  'n**r': 'nigger',
                  'n*r': 'nigger',
                  'a*****': 'ass',
                  'a****': 'ass',
                  'a***': 'ass',
                  'a**': 'ass',
                  'a*': 'ass',
                  'c********': 'cunt',
                  'c*******': 'cunt',
                  'c******': 'cunt',
                  'c*****': 'cunt',
                  'c****': 'cunt',
                  'c***': 'cunt',
                  'c**': 'cunt',
                  'c*': 'cunt',
                  'c********t': 'cunt',
                  'c*******t': 'cunt',
                  'c******t': 'cunt',
                  'c*****t': 'cunt',
                  'c****t': 'cunt',
                  'c***t': 'cunt',
                  'c**t': 'cunt',
                  'c*t': 'cunt',
                  'wtf': 'what the fuck',
                  ' u ': 'you',
                  'yrs': 'years',
                  }
    tweet = tweet.lower()
    for hidden, replacement in hidden_dic.items():
      tweet = tweet.replace(hidden, ' ' + replacement + ' ')
    
    tweet = re.sub(r"\x89Û_", "", tweet)
    tweet = re.sub(r"\x89ÛÒ", "", tweet)
    tweet = re.sub(r"\x89ÛÓ", "", tweet)
    tweet = re.sub(r"\x89ÛÏWhen", "When", tweet)
    tweet = re.sub(r"\x89ÛÏ", "", tweet)
    tweet = re.sub(r"China\x89Ûªs", "China's", tweet)
    tweet = re.sub(r"let\x89Ûªs", "let's", tweet)
    tweet = re.sub(r"\x89Û÷", "", tweet)
    tweet = re.sub(r"\x89Ûª", "", tweet)
    tweet = re.sub(r"\x89Û\x9d", "", tweet)
    tweet = re.sub(r"å_", "", tweet)
    tweet = re.sub(r"\x89Û¢", "", tweet)
    tweet = re.sub(r"\x89Û¢åÊ", "", tweet)
    tweet = re.sub(r"fromåÊwounds", "from wounds", tweet)
    tweet = re.sub(r"åÊ", "", tweet)
    tweet = re.sub(r"åÈ", "", tweet)
    tweet = re.sub(r"JapÌ_n", "Japan", tweet)    
    tweet = re.sub(r"Ì©", "e", tweet)
    tweet = re.sub(r"å¨", "", tweet)
    tweet = re.sub(r"SuruÌ¤", "Suruc", tweet)
    tweet = re.sub(r"åÇ", "", tweet)
    tweet = re.sub(r"å£3million", "3 million", tweet)
    tweet = re.sub(r"åÀ", "", tweet)
    
    # Contractions
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"We're", "We are", tweet)
    tweet = re.sub(r"That's", "That is", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"Can't", "Cannot", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"don\x89Ûªt", "do not", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"What's", "What is", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"There's", "There is", tweet)
    tweet = re.sub(r"He's", "He is", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"You're", "You are", tweet)
    tweet = re.sub(r"I'M", "I am", tweet)
    tweet = re.sub(r"shouldn't", "should not", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"i'm", "I am", tweet)
    tweet = re.sub(r"I\x89Ûªm", "I am", tweet)
    tweet = re.sub(r"I'm", "I am", tweet)
    tweet = re.sub(r"Isn't", "is not", tweet)
    tweet = re.sub(r"Here's", "Here is", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you\x89Ûªve", "you have", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"it\x89Ûªs", "it is", tweet)
    tweet = re.sub(r"doesn\x89Ûªt", "does not", tweet)
    tweet = re.sub(r"It\x89Ûªs", "It is", tweet)
    tweet = re.sub(r"Here\x89Ûªs", "Here is", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"I\x89Ûªve", "I have", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"can\x89Ûªt", "cannot", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"wouldn\x89Ûªt", "would not", tweet)
    tweet = re.sub(r"We've", "We have", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"Y'all", "You all", tweet)
    tweet = re.sub(r"Weren't", "Were not", tweet)
    tweet = re.sub(r"Didn't", "Did not", tweet)
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"they'd", "they would", tweet)
    tweet = re.sub(r"DON'T", "DO NOT", tweet)
    tweet = re.sub(r"That\x89Ûªs", "That is", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"You\x89Ûªre", "You are", tweet)
    tweet = re.sub(r"where's", "where is", tweet)
    tweet = re.sub(r"Don\x89Ûªt", "Do not", tweet)
    tweet = re.sub(r"we'd", "we would", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"They're", "They are", tweet)
    tweet = re.sub(r"Can\x89Ûªt", "Cannot", tweet)
    tweet = re.sub(r"you\x89Ûªll", "you will", tweet)
    tweet = re.sub(r"I\x89Ûªd", "I would", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"can't", "cannot", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"i've", "I have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"ain't", "am not", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"I've", "I have", tweet)
    tweet = re.sub(r"Don't", "do not", tweet)
    tweet = re.sub(r"I'll", "I will", tweet)
    tweet = re.sub(r"I'd", "I would", tweet)
    tweet = re.sub(r"Let's", "Let us", tweet)
    tweet = re.sub(r"you'd", "You would", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"Ain't", "am not", tweet)
    tweet = re.sub(r"Haven't", "Have not", tweet)
    tweet = re.sub(r"Could've", "Could have", tweet)
    tweet = re.sub(r"youve", "you have", tweet)  
    tweet = re.sub(r"donå«t", "do not", tweet)   
            
    # Character entity references
    tweet = re.sub(r"&gt;", ">", tweet)
    tweet = re.sub(r"&lt;", "<", tweet)
    tweet = re.sub(r"&amp;", "&", tweet)
    
    return tweet


def preprocess(text):
    custom_replace = {'’': '\'', '\'t': ' not'}
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    for hidden, replacement in custom_replace.items():
      text = text.replace(hidden, replacement)

    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    return ' '.join(text.split())




train_df['tweet'] = train_df['tweet'].apply(replace_hidden)
train_df['tweet'] = train_df['tweet'].apply(preprocess)

test_df['tweet'] = test_df['tweet'].apply(replace_hidden)
test_df['tweet'] = test_df['tweet'].apply(preprocess)

train_df.head()

,Tweet_ID,tweet,type
23638,ID_LK3FHU9H,they said something along the lines of this i know him he wouldn not do something like that as well as telling one of my bestfriends that they didn not believe me because i just wanted dick and i wouldn not want it if i was raped men suck,sexual_violence
18880,ID_H6OK5R5D,god did fuck all to do with children starving and people getting raped and murdered for thousands of years and you say he created me how about he creates a happy planet with good people fuck your god he can eat shit,sexual_violence
34891,ID_VRRYJ4Z3,oh let us count the questions that donnie trump must answer 1 how does he get by paying 0 to 750 in income taxes when most average americans pay about 10 times that in any given year 2 how many women has donnie raped,sexual_violence
1015,ID_0VJU7B2W,fgm female genital mutilation includes the partial total removal of the female external genitalia causing other injuries to the female genital organs for non medical reasons why i broke the law to undergo fgm aged 26,Harmful_Traditional_practice
28350,ID_PUR1YBUB,my husband always beats me at scrabble,Physical_violence


In [ ]:
type_to_id = {'sexual_violence': 0,
              'Physical_violence': 1,
              'emotional_violence': 2,
              'Harmful_Traditional_practice': 3,
              'economic_violence': 4,
              }

types = ['sexual_violence', 'Physical_violence', 'emotional_violence', 'Harmful_Traditional_practice', 'economic_violence']

def encode_label(text):
    return type_to_id[text]

train_df['type'] = train_df['type'].apply(encode_label)

In [ ]:
tasks = [
         'hate',
         'offensive',
         'emotion',
        #  'irony',
        #  'sentiment'
         ]
bert_paths = []

for task in tasks:
  path = f"cardiffnlp/twitter-roberta-base-{task}"
  bert_paths.append(path)

bert_paths

['cardiffnlp/twitter-roberta-base-hate',
 'cardiffnlp/twitter-roberta-base-offensive',
 'cardiffnlp/twitter-roberta-base-emotion']

In [ ]:
seq_len = 120

berts = []
tokenizers = []

for path in bert_paths:
    tokenizer = AutoTokenizer.from_pretrained(path)
    bert = TFAutoModel.from_pretrained(path)
    tokenizers.append(tokenizer)
    berts.append(bert)



Some layers from the model checkpoint at cardiffnlp/twitter-roberta-base-hate were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-hate.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at cardiffnlp/twitter-roberta-base-offensive were not used when initializing TFRober

Some layers from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion were not used when initializing TFRobertaModel: ['classifier']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
token_data = []

for tokenizer in tokenizers:
    tokens = tokenizer(train_df['tweet'].tolist(), max_length=seq_len, 
                      truncation=True, padding='max_length', 
                      add_special_tokens=True, return_tensors='np')
    token_data.append(tokens)


In [ ]:
token_ids_data = []
token_mask_data = []


for tok in token_data:
  tokens_id = tok['input_ids']
  tokens_mask = tok['attention_mask']
  token_ids_data.append(tokens_id)
  token_mask_data.append(tokens_mask)


y_values = train_df['type'].to_numpy()


In [ ]:
y_train = y_values
train_data = {}
for i, task in enumerate(tasks):
    train_data[task+'_input_ids'] = token_ids_data[i]
    train_data[task+'_attention_mask'] = token_mask_data[i]
  
train_data.keys()

dict_keys(['hate_input_ids', 'hate_attention_mask', 'offensive_input_ids', 'offensive_attention_mask', 'emotion_input_ids', 'emotion_attention_mask'])

In [ ]:
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE
BUFFER_SIZE = 50000


train_ds = Dataset.from_tensor_slices((train_data, y_train)).shuffle(buffer_size=BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)


In [ ]:
def create_model():
    input_ids = [Input(shape=(seq_len,), name=task+'_input_ids', dtype='int32') for task in tasks]
    masks = [Input(shape=(seq_len,), name=task+'_attention_mask', dtype='int32') for task in tasks]

    encoded = [bert(trf_input_ids, attention_mask=trf_mask)[0] for trf_input_ids, trf_mask in zip(input_ids, masks)]
    encoded = [encd[:, 0, :] for encd in encoded]
    output = [Dense(512, activation='relu')(encd) for encd in encoded]
    output = concatenate(output)
    output = Dense(5, activation='softmax')(output)

    inputs = [ids for ids in input_ids] + [mask for mask in masks]
    return Model(inputs=inputs, outputs=[output])

model = create_model()

In [ ]:
INDEX = len(tasks)*2
model.layers[INDEX].trainable = False

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
hate_input_ids (InputLayer)     [(None, 120)]        0                                            
__________________________________________________________________________________________________
hate_attention_mask (InputLayer [(None, 120)]        0                                            
__________________________________________________________________________________________________
offensive_input_ids (InputLayer [(None, 120)]        0                                            
__________________________________________________________________________________________________
offensive_attention_mask (Input [(None, 120)]        0                                            
____________________________________________________________________________________________

In [ ]:
weights = compute_class_weight(class_weight='balanced', classes=[0, 1, 2, 3, 4], y=train_df['type'])
class_weight ={}

for i in range(len(weights)):
    class_weight[i] = weights[i]

In [ ]:
import math

def step_decay(epoch):
	initial_lrate = 0.00125
	drop = 0.5
	epochs_drop = 2.0
	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lrate

lr = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.fit(train_ds, epochs=3, class_weight=class_weight)


Epoch 1/3
619/619 [==============================] - 316s 475ms/step - loss: 0.1830 - accuracy: 0.9345
Epoch 2/3
619/619 [==============================] - 294s 475ms/step - loss: 0.1585 - accuracy: 0.9311
Epoch 3/3
619/619 [==============================] - 295s 476ms/step - loss: 0.2025 - accuracy: 0.9311


In [ ]:
import json

weights = {}
weights['layer14'] = model.layers[-1].get_weights()[0].tolist()
weights['layer10'] = model.layers[10].get_weights()[0].tolist()
weights['layer11'] = model.layers[11].get_weights()[0].tolist()
weights['layer12'] = model.layers[12].get_weights()[0].tolist()
weights['layer14_b'] = model.layers[-1].get_weights()[1].tolist()
weights['layer10_b'] = model.layers[10].get_weights()[1].tolist()
weights['layer11_b'] = model.layers[11].get_weights()[1].tolist()
weights['layer12_b'] = model.layers[12].get_weights()[1].tolist()

with open('/content/drive/MyDrive/chekcpoint.json', 'w') as f:
    json.dump(weights, f)
f.close()

In [ ]:
test_token_data = []
test_data = {}
test_token_ids_data = []
test_token_mask_data = []


for tokenizer in tokenizers:
    tokens = tokenizer(test_df['tweet'].tolist(), max_length=seq_len, 
                      truncation=True, padding='max_length', 
                      add_special_tokens=True, return_tensors='np')
    test_token_data.append(tokens)


for tok in test_token_data:
    test_tokens_id = tok['input_ids']
    test_tokens_mask = tok['attention_mask']
    test_token_ids_data.append(test_tokens_id)
    test_token_mask_data.append(test_tokens_mask)


for i, task in enumerate(tasks):
    test_data[task+'_input_ids'] = test_token_ids_data[i]
    test_data[task+'_attention_mask'] = test_token_mask_data[i]
  
test_data.keys()

dict_keys(['hate_input_ids', 'hate_attention_mask', 'offensive_input_ids', 'offensive_attention_mask', 'emotion_input_ids', 'emotion_attention_mask'])

In [ ]:
predicted = model.predict(test_data)

In [ ]:
predictions = []
for p in predicted:
    predictions.append(np.argmax(p))


In [ ]:
simple_submission = pd.DataFrame()
simple_submission['Tweet_ID'] = test_df['Tweet_ID']
simple_submission['type'] = predictions
simple_submission['type'] = simple_submission['type'].apply(lambda x: types[x])
simple_submission.head(20)

,Tweet_ID,type
0,ID_0095QL4S,Physical_violence
1,ID_00DREW5O,Harmful_Traditional_practice
2,ID_00E9F5X9,Harmful_Traditional_practice
3,ID_00G9OSKZ,sexual_violence
4,ID_00HU96U6,sexual_violence
5,ID_00IJ4SAW,sexual_violence
6,ID_00J80GL8,Physical_violence
7,ID_00L0E289,emotional_violence
8,ID_00N24MZN,sexual_violence
9,ID_00V2TOUW,sexual_violence


In [ ]:
simple_submission.to_csv('simple_submission.csv', index=False)

In [ ]:
from google.colab import files

files.download('/content/simple_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>